In [1]:
// 📦 Import x402 Buyer Dependencies
// Following official Quickstart Guide exactly!

import { x402Client, wrapFetchWithPayment, x402HTTPClient } from "npm:@x402/fetch@^2.0.0";
import { registerExactEvmScheme } from "npm:@x402/evm@^2.0.0/exact/client";

// Viem for wallet and chains
import { privateKeyToAccount } from "npm:viem@^2.0.0/accounts";
import { optimism, optimismSepolia } from "npm:viem@^2.0.0/chains";

// Load environment variables
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
const env = await load({ export: true });

console.log("✅ Dependencies loaded!");
console.log("   x402Client:", typeof x402Client);
console.log("   wrapFetchWithPayment:", typeof wrapFetchWithPayment);
console.log("   registerExactEvmScheme:", typeof registerExactEvmScheme);
console.log("   Chains:", { optimism: optimism.name, optimismSepolia: optimismSepolia.name });

✅ Dependencies loaded!
   x402Client: function
   wrapFetchWithPayment: function
   registerExactEvmScheme: function
   Chains: { optimism: "OP Mainnet", optimismSepolia: "OP Sepolia" }


## 🔧 Konfiguration

Alle Werte werden aus der `.env` Datei geladen:
- **PRIVATE_KEY**: Dein Wallet Private Key
- **SERVICE_URL**: Dein GenImg Service Endpoint
- **FACILITATOR_URL**: Dein x402 Facilitator

**🔍 Wie x402 v2 Network-Auswahl funktioniert:**

1. **Server bietet Networks an**: Server sendet unterstützte Networks im 402-Response (`accepts` array)
2. **Client wählt aus**: Default = erstes Network im `accepts` array
3. **Server kontrolliert**: Via `sepoliaTest` Flag im Request Body

**Server-Side Network Control:**
- `sepoliaTest: true` → Server bietet nur Sepolia an
- `sepoliaTest: false` → Server bietet alle Networks an (Sepolia zuerst)

Client nimmt automatisch das erste angebotene Network - einfach und funktioniert!

**Testnet:** Optimism Sepolia (USDC: 0x5fd8...)  
  - ⚠️ **GenImg Contract NICHT deployed** - Pre-flight checks werden fehlschlagen!

**Mainnet:** Optimism (USDC: 0x0b2C...)  
  - ✅ GenImg Contract deployed (0x80f95d330417a4acEfEA415FE9eE28db7A0A1Cdb)
  - ⚠️ **ECHTES GELD!**

In [2]:
// ⚙️ Configuration - lädt aus .env Datei
const PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const SERVICE_URL = "http://localhost:8082";
const FACILITATOR_URL = "https://facilitator.fretchen.eu";
const PROMPT = "A cute robot painting a picture";

// ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
const USE_MAINNET = false;  // Set to true for Optimism Mainnet with REAL MONEY

// Network configuration based on selection
const NETWORK = USE_MAINNET ? {
  // Optimism Mainnet - REAL MONEY! 💰
  chain: optimism,
  chainId: "eip155:10",
  name: "Optimism Mainnet",
  usdc: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
  decimals: 6,
} : {
  // Optimism Sepolia - Testnet 🧪
  chain: optimismSepolia,
  chainId: "eip155:11155420",
  name: "Optimism Sepolia",
  usdc: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
  decimals: 6,
};

// Show configuration
console.log("📋 Configuration:");
console.log(`   PRIVATE_KEY: ${PRIVATE_KEY ? PRIVATE_KEY.slice(0, 6) + "..." + PRIVATE_KEY.slice(-4) : "❌ NOT FOUND"}`);
console.log(`   SERVICE_URL: ${SERVICE_URL}`);
console.log(`   FACILITATOR_URL: ${FACILITATOR_URL}`);

if (USE_MAINNET) {
  console.log(`\n🚨 WARNING: Using REAL MONEY on ${NETWORK.name}!`);
  console.log(`⚠️  Make sure this is intentional!`);
} else {
  console.log(`\n🧪 Using testnet: ${NETWORK.name}`);
}

console.log(`\n📍 Network: ${NETWORK.name} (${NETWORK.chainId})`);
console.log(`💰 USDC Contract: ${NETWORK.usdc}`);
console.log(`🎨 Prompt: "${PROMPT}"`);

📋 Configuration:
   PRIVATE_KEY: 3572d2...446f
   SERVICE_URL: http://localhost:8082
   FACILITATOR_URL: https://facilitator.fretchen.eu

🧪 Using testnet: Optimism Sepolia

📍 Network: Optimism Sepolia (eip155:11155420)
💰 USDC Contract: 0x5fd84259d66Cd46123540766Be93DFE6D43130D7
🎨 Prompt: "A cute robot painting a picture"


## 🔑 Setup x402 Client (Quickstart Style)

Minimales Setup - genau wie im offiziellen Quickstart!

In [ ]:
// 🔧 Setup x402 client - Exactly like Quickstart!

console.log("🔧 Setting up x402 client...");

// Create signer (wallet)
const signer = privateKeyToAccount(`0x${PRIVATE_KEY}`);

// Create x402 client
const client = new x402Client();

// Register EVM scheme (tells client how to create EIP-3009 signatures)
registerExactEvmScheme(client, { signer });

// Wrap fetch with payment handling
const fetchWithPayment = wrapFetchWithPayment(fetch, client);

console.log("✅ x402 client ready");
console.log("   Will use server's first offered network");
console.log("   Server controls via sepoliaTest flag");

🔧 Setting up x402 client...


TypeError: Cannot read properties of undefined (reading 'networks')

## 🚀 Execute Payment Flow with x402Fetch

**Das war's!** Ein einfacher fetch - x402Fetch macht alles automatisch:
1. Initial request → 402 Response
2. Parse Payment Requirements
3. Create EIP-3009 Signature
4. Retry mit Payment
5. Success! ✅


## 🔍 Pre-Request: Verify Contract Deployment

**WICHTIG:** Der GenImg Contract ist nur auf Optimism Mainnet deployed!
Diese Zelle prüft ob der Contract auf dem gewählten Network existiert:

## 🐛 DEBUG: Intercept Payment Network

**Führe diese Zelle aus BEVOR du den Request machst!**
Sie zeigt dir welches Network tatsächlich im Payment-Payload steht:

In [ ]:
// 🚀 Make paid request
// (x402HTTPClient already imported in Cell 1)

console.log("🚀 Making paid request...");

// Make request - payment is handled automatically (exactly like Quickstart!)
const response = await fetchWithPayment(`${SERVICE_URL}`, {
  method: "POST",
  headers: {
    "Content-Type": "application/json",
  },
  body: JSON.stringify({ 
    prompt: PROMPT,
    sepoliaTest: !USE_MAINNET  // Server will restrict to Sepolia if true
  }),
});

console.log(`📡 Response Status: ${response.status}`);

if (response.ok) {
  const result = await response.json();
  console.log("\n🎉 SUCCESS!");
  console.log(`🖼️  Image URL: ${result.imageUrl}`);
  console.log(`🎫 NFT Token ID: ${result.tokenId}`);
  console.log(`📍 Contract: ${result.contractAddress}`);
  
  // Get payment receipt from response headers (exactly like Quickstart!)
  const httpClient = new x402HTTPClient(client);
  const paymentResponse = httpClient.getPaymentSettleResponse(
    (name) => response.headers.get(name)
  );
  
  if (paymentResponse) {
    console.log("\n✅ Payment settled:");
    console.log(`   Transaction: ${paymentResponse.transaction}`);
    console.log(`   Network: ${paymentResponse.network}`);
  }
  
  console.log("\n💡 fetchWithPayment automatically:");
  console.log("   1. Received 402 Payment Required");
  console.log("   2. Parsed payment requirements");
  console.log("   3. Created EIP-3009 signature");
  console.log("   4. Retried with payment");
  console.log("   5. Got successful response!");
} else {
  const error = await response.text();
  console.error(`❌ Error: ${error}`);
  throw new Error(`Request failed: ${response.status}`);
}

🚀 Making paid request...


Error: Failed to create payment payload: No client registered for x402 version: 2

## 🎨 Display Generated Image

Wenn erfolgreich, kannst du das Bild hier anzeigen:

In [ ]:
// TODO: Nach erfolgreichem Flow die imageUrl hier einfügen
const imageUrl = "https://..."; // From previous cell result

if (imageUrl) {
  // In Deno notebooks können wir HTML direkt rendern
  const html = `
    <div style="text-align: center; padding: 20px;">
      <h3>Generated Image</h3>
      <img src="${imageUrl}" alt="Generated Image" style="max-width: 100%; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);" />
    </div>
  `;
  
  // Display HTML (funktioniert in Jupyter mit Deno)
  await $`echo ${html}`;
}

## 📊 Debug: Check USDC Balance

Prüfe deine USDC Balance vor und nach dem Payment:

In [ ]:
import { createPublicClient } from "npm:viem@^2.0.0";
import { erc20Abi } from "npm:viem@^2.0.0";

const publicClient = createPublicClient({
  chain: NETWORK.chain,
  transport: http(),
});

const balance = await publicClient.readContract({
  address: NETWORK.usdc,
  abi: erc20Abi,
  functionName: "balanceOf",
  args: [account.address],
});

const balanceFormatted = parseFloat(balance.toString()) / 10 ** NETWORK.decimals;

console.log(`💰 USDC Balance: ${balanceFormatted} USDC`);
console.log(`📍 Network: ${NETWORK.name}`);
console.log(`🔑 Address: ${account.address}`);

## 🔍 Debug: Query Facilitator Supported Networks

Prüfe welche Networks der Facilitator unterstützt:

In [ ]:
const supportedResponse = await fetch(`${FACILITATOR_URL}/supported`);
const supported = await supportedResponse.json();

console.log("🌐 Facilitator Supported Networks:");
console.log(JSON.stringify(supported, null, 2));

// Check if our network is supported
const ourNetworkSupported = supported.kinds?.some(
  (kind) => kind.network === NETWORK.chainId && kind.scheme === "exact"
);

console.log(`\n✅ ${NETWORK.name} supported: ${ourNetworkSupported}`);

## 🧪 Advanced: Manual Payment Creation

Falls du den Payment Flow manuell testen möchtest (ohne x402Client):

In [ ]:
// Manual EIP-3009 Transfer Signature Creation
import { keccak256, encodeAbiParameters, parseAbiParameters } from "npm:viem@^2.0.0";

async function createManualPayment(
  from: string,
  to: string,
  amount: bigint,
  validAfter: bigint = 0n,
  validBefore: bigint = BigInt(Math.floor(Date.now() / 1000) + 3600)
) {
  // Generate nonce (random 32 bytes)
  const nonce = "0x" + Array.from(crypto.getRandomValues(new Uint8Array(32)))
    .map(b => b.toString(16).padStart(2, '0'))
    .join('');

  // EIP-3009 Transfer Authorization
  const domain = {
    name: "USD Coin",
    version: "2",
    chainId: NETWORK.chain.id,
    verifyingContract: NETWORK.usdc,
  };

  const types = {
    TransferWithAuthorization: [
      { name: "from", type: "address" },
      { name: "to", type: "address" },
      { name: "value", type: "uint256" },
      { name: "validAfter", type: "uint256" },
      { name: "validBefore", type: "uint256" },
      { name: "nonce", type: "bytes32" },
    ],
  };

  const message = {
    from,
    to,
    value: amount,
    validAfter,
    validBefore,
    nonce,
  };

  // Sign with wallet
  const signature = await walletClient.signTypedData({
    account,
    domain,
    types,
    primaryType: "TransferWithAuthorization",
    message,
  });

  return {
    authorization: {
      v: parseInt(signature.slice(130, 132), 16),
      r: "0x" + signature.slice(2, 66),
      s: "0x" + signature.slice(66, 130),
      validAfter: validAfter.toString(),
      validBefore: validBefore.toString(),
      nonce,
    },
    transfer: {
      from,
      to,
      value: amount.toString(),
    },
  };
}

console.log("✅ Manual payment creation function defined");
console.log("   Use: await createManualPayment(from, to, amount)");

## 📝 Notes & Troubleshooting

### Häufige Fehler:

1. **"Network not supported"**
   - Prüfe ob Service und Facilitator das gewählte Network unterstützen
   - Check `/supported` endpoint vom Facilitator

2. **"Insufficient USDC balance"**
   - Hole Testnet USDC von Faucet (für Sepolia)
   - Oder kaufe USDC auf Mainnet

3. **"Payment verification failed"**
   - Prüfe ob Facilitator läuft
   - Check Signature Format (v2 vs v1)

4. **"Transaction reverted"**
   - USDC Approve fehlt eventuell
   - Check Nonce conflicts

### Nützliche Links:

- [x402 v2 Docs](https://github.com/coinbase/x402)
- [Optimism Sepolia Faucet](https://faucet.optimism.io/)
- [Base Sepolia Faucet](https://www.coinbase.com/faucets/base-ethereum-goerli-faucet)
- [USDC Contracts](https://developers.circle.com/stablecoins/docs/usdc-on-test-networks)